In [ ]:
import pandas as pd
import cufflinks as cf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Object and Emotion detection - without bounding boxes

This code snippet extract object and emotions from the image set

In [ ]:
import os
from PIL import Image
import glob
import pandas as pd
import numpy as np
from deepface import DeepFace
import torch

path_model = r"path"
model = torch.hub.load('ultralytics/yolov5', 'custom', path=path_model)

path_to_images = r"Images"

# Get a list of image file paths
image_paths = glob.glob(os.path.join(path_to_images, "*.png"))

# Create an empty dictionary to store the results
results = {}

# Loop over each image in the folder
for image_path in image_paths:
    
    # Load the image
    try:
        img = Image.open(image_path)
    except:
        results[os.path.basename(image_path)] = {'object': None, 'dominant_emotion': None}
        continue
    
    # Convert image to numpy array
    img = np.array(img)
    
    # Use YOLOv5 to detect objects in the image
    results_yolo = model(img)
    
    # Get a list of detected objects
    detections = results_yolo.pandas().xyxy[0]
    
    # Check if a person was detected
    person_detected = False
    for detection in detections.iterrows():
        if detection[1]['name'] == 'person':
            person_detected = True
            break
    
    # If a person was detected, deepface library is used to extract dominant emotion from image
    if person_detected:
        emotions = DeepFace.analyze(img, actions = ['emotion'], enforce_detection=False)
        dominant_emotion = emotions['dominant_emotion']
        object_detected = 'person'
    # If no person was detected, set object_detected to the name of the first object detected (if any)
    else:
        if len(detections) > 0:
            object_detected = detections.iloc[0]['name']
#         else:
            object_detected = None
        dominant_emotion = None
    
    # Store the results in the dictionary
    results[os.path.basename(image_path)] = {'object': object_detected, 'dominant_emotion': dominant_emotion}

# Convert the dictionary to a pandas DataFrame
df_results = pd.DataFrame.from_dict(results, orient='index')

#Removing unnecessary .png from the image folder
df_results['CreativeId'] = df_results.index.str.replace('.png', '').astype(np.int64)

#Loading the dataset created in the EDA-file
emotion_results = pd.read_csv('emotion_results.csv')

# Merge the two DataFrames based on the CreativeId column
final_df = pd.merge(emotion_results, df_results[['CreativeId', 'object', 'dominant_emotion']], on='CreativeId', how='left')

# Save the final DataFrame so it can be used for further analysis
final_df.to_csv('final_results.csv', index=False)

# Object detection with only objects and their bounding boxes

In [ ]:
import os
import torch
from PIL import Image
import numpy as np
import pandas as pd

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

path_to_image_folder = r'Images'

image_files = os.listdir(path_to_image_folder)

obj_results = pd.DataFrame(columns=['id', 'label', 'confidence', 'x_min', 'y_min', 'x_max', 'y_max'])
for image_file in image_files:
    image_path = os.path.join(path_to_image_folder, image_file)
    try:
        im = Image.open(image_path)
    except:
        continue
    results = model(np.array(im))
    detections = results.xyxy[0]
    for i in range(len(detections)):
        label = results.names[int(detections[i][5])]
        confidence = detections[i][4]
        x_min, y_min, x_max, y_max = detections[i][:4]
        obj_results = pd.concat([df_results, pd.DataFrame({'id': [image_file], 'label': [label], 'confidence': [confidence],
                                        'x_min': [x_min], 'y_min': [y_min], 'x_max': [x_max], 'y_max': [y_max]})])
        

obj_results.to_csv('final_results_objects.csv', index=False)



# Preparing data

In this code snippet the results from the object detection and ground truth tables are prepared so they can be evaluated

In [ ]:
import tensorflow as tf
import numpy as np

# Load the ground truth table
objects_detected = pd.read_csv('final_results_objects.csv')
ground_truth= pd.read_csv('labels_gr-table.csv')

import re

#Helping method to only extract the numeric values(needed as there was string values in the columns)
def extract_float_from_tensor(tensor_str):
    match = re.search(r"tensor\((\d+\.\d+)\)", tensor_str)
    if match:
        return float(match.group(1))
    else:
        return None

objects_detected[['confidence', 'x_min', 'y_min', 'x_max', 'y_max']] = objects_detected[['confidence', 'x_min', 'y_min', 'x_max', 'y_max']].applymap(extract_float_from_tensor)

# Load the final results of your model
emotions = ['happy', 'surprise', 'fear', 'angry', 'sad']
ground_truth = ground_truth[~ground_truth['label_name'].isin(emotions)]
ground_truth = ground_truth.rename(columns={'image_name': 'id'})  # rename the column



# IoU

In this code snippet intersection over union is utilized to evaluate the ground truth table with the object detection model sing bounding boxes

In [ ]:
# Connecting the object detection results and ground truth results together
evaluate = pd.merge(objects_detected, ground_truth, on='id')

# Calculate the area of the intersection between the bounding boxes using the formula for intersection over union
evaluate['intersection_area'] = (evaluate[['x_max', 'bbox_x']].min(axis=1) - evaluate[['x_min', 'bbox_x']].max(axis=1)) * \
                                  (evaluate[['y_max', 'bbox_y']].min(axis=1) - evaluate[['y_min', 'bbox_y']].max(axis=1))
evaluate['intersection_area'] = evaluate['intersection_area'].clip(lower=0)

# Calculate the area of the union between the bounding boxes
evaluate['union_area'] = (evaluate['x_max'] - evaluate['x_min']) * (evaluate['y_max'] - evaluate['y_min']) + \
                          (evaluate['bbox_width'] * evaluate['bbox_height']) - evaluate['intersection_area']

# Calculate the intersection over union (IoU) score
evaluate['iou'] = evaluate['intersection_area'] / evaluate['union_area']

# Group by label name and calculate the mean IoU score for each label
mean_iou_scores = evaluate.groupby('label_name')['iou'].mean()
mean_iou_scores



# Loading the dataset with object and emotion detection - withouth bounding boxes

In [ ]:
obj_emo_results = pd.read_csv('final_results.csv')

obj_emo_results['object'].fillna('no object', inplace=True)
obj_emo_results['dominant_emotion'].fillna('not_person', inplace=True)

#CreativeId (advertisement's id) since it is no longer important, and will disturb further analysis with ML-models
obj_emo_results.drop("CreativeId", axis=1)



# Highest CTR-value for object and emotion

In [ ]:
sorted_obj_emo = obj_emo_results.sort_values(by='ctr', ascending=False)

# Find the most popular objects based on CTR
top_objects = sorted_obj_emo.groupby('object')['ctr'].mean().nlargest(5)


# Find the most popular dominant emotions based on CTR
top_emotions = sorted_obj_emo.groupby('dominant_emotion')['ctr'].mean().nlargest(5)
merged_obj_emo = pd.concat([top_objects, top_emotions], axis=0)
merged_obj_emo

# Distribution for emotions and objects

In [ ]:
objects = obj_emo_results["object"].value_counts()
emotions = obj_emo_results["dominant_emotion"].value_counts()
# Bar chart of object counts
plt.bar(objects.index, objects.values, color=['blue', 'red', 'green', 'orange'])
plt.title("Object Counts")
plt.xlabel("Object")
plt.ylabel("Count")
plt.xticks(rotation=45)
for i, v in enumerate(objects.values):
    plt.text(i, v, str(v), color='black', fontweight='bold', ha='center')
plt.savefig('objectsdistribution.png', dpi=300, bbox_inches='tight')

plt.show()

# Bar chart of emotion counts
plt.bar(emotions.index, emotions.values, color=['red', 'blue', 'green', 'orange', 'purple', 'brown'])
plt.title("Emotion Counts")
plt.xlabel("Emotion")
plt.ylabel("Count")
plt.xticks(rotation=45)
for i, v in enumerate(emotions.values):
    plt.text(i, v, str(v), color='black', fontweight='bold', ha='center')
plt.savefig('emotionsdistribution.png', dpi=300, bbox_inches='tight')

plt.show()

# Making copy of data set to build further on

In [ ]:
df_updated = obj_emo_results.copy()

# Preprocessing of the data

In order to be able to perfom machine-learning models, some preprocessing of the data var neccesary to get appropriate data-values.

In [ ]:
from sklearn.preprocessing import LabelEncoder

df_transform = df_updated.copy()

# Encode the age_group variable
age_encoder = LabelEncoder()
df_transform['age_group'] = age_encoder.fit_transform(df_transform['age_group'])

# Encode the gender variable
gender_encoder = LabelEncoder()
df_transform['gender'] = gender_encoder.fit_transform(df_transform['gender'])

# Convert n_click and n_impressions_measurable to integers
df_transform['n_click'] = df_transform['n_click'].astype(int)
df_transform['n_impressions_measurable'] = df_transform['n_impressions_measurable'].astype(int)

# Categorical variables into integers using LabelEncoder
categorical = ['Adviser', 'industry', 'Category', 'page_type', 'format', 'object', 'dominant_emotion']
for col in categorical:
    label_encoder = LabelEncoder()
    df_transform[col] = label_encoder.fit_transform(df_transform[col])

# Predictive models

Random forest model, using gridsearch to find best parameters

In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# Random Forest Classifier
# Split the data into training and test sets, n_click as target variable
X_train, X_test, y_train, y_test = train_test_split(df_transform.drop('n_click', axis=1), df_transform['n_click'], test_size=0.2, random_state=42)

# The parameters under is obtained from the gridsearch
rfc = RandomForestRegressor(n_estimators=50, max_features=0.8, max_samples=0.8, max_depth=9, min_samples_split=5, random_state=42)


# Fit the model to the training set
rfc.fit(X_train, y_train)

# Predict the test set
y_pred = rfc.predict(X_test)



# # Remove comment to perform the grid search
# parameters = {
#     'n_estimators':np.arange(50,200,15),
#               'max_features':np.arange(0.1, 1, 0.1),
#               'max_depth': [3, 5, 7, 9],
#               'max_samples': [0.3, 0.5, 0.8]
# }

# dtc = RandomForestClassifier(random_state=42)
# clf = GridSearchCV(dtc, parameters, cv=3)
# clf.fit(X_train, y_train)
# paramstouse = clf.best_params_
# print(f"Most suitable parameters for the classifier: {paramstouse}")


# Compute evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Random Forest Regressor")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R2 Score: {r2:.2f}")
# Calculate feature importances
importances = rfc.feature_importances_
sorted_indices = np.argsort(importances)[::-1]
features = X_train.columns

# Print feature importances
print('\nFeature ranking:')
for i, idx in enumerate(sorted_indices):
    print(f"{i+1}. {features[idx]} ({importances[idx]:.6f})")


# Baseline

Decision tree built as baseline

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(random_state=42)

tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

r2 = r2_score(y_test, y_pred)

print("Performance of the Decision Tree Regressor")

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R^2): {r2:.2f}")


#feature importances
importances = tree.feature_importances_
sorted_indices = np.argsort(importances)[::-1]
features = X_train.columns
print('Feature ranking:')
for i, idx in enumerate(sorted_indices):
    print(f"{i+1}. {features[idx]} ({importances[idx]:.6f})")